# `nmtools ufunc` Examples

This notebook shows you various built-in ufuncs in nmtools and several custom ufuncs.
The term "ufunc" is short for "universal function" from numpy's [ufunc](https://numpy.org/doc/stable/reference/ufuncs.html).
nmtools' ufunc is directly inspired from that numpy's ufunc, contains several generic operations:
- the ufunc: arbitrary arity functions with broadcasting.
- reduce ufunc: reducing the array dimension by applying function along the given axis (or axes).
- outer ufunc: apply the function to each pair consists of elements of A and B.
- accumulate ufunc: accumulate the result from given operation along an axis.

Unlike numpy, nmtools doesn't provide the following ufunc:
- reduceat
- at
- generalized ufunc

Most of numpy's ufuncs are also available in nmtools.

To use this notebook, you must install xeus-cling cpp jupyter kernel to be able to use C++17 on notebook. For more info about xeus-cling please refer to https://github.com/jupyter-xeus/xeus-cling.

nmtools is a header-only library, no library to link to, you only need to provide include path, such as the following:

In [1]:
#pragma cling add_include_path("/workspaces/numeric_tools/include")

Here we include all necessary nmtools header and to reduce verbosity, declare namespace alias and some using-directives.

Note that `None`, `True`, `False` are nmtools constants. The statement `using namespace nm::literals` is to enable nmtools UDL (`_ct`).

In [2]:
#include "nmtools/array/shape.hpp"
#include "nmtools/array/array/ufuncs/add.hpp"
#include "nmtools/array/array/ufuncs/multiply.hpp"
#include "nmtools/array/array/ufuncs/clip.hpp"
#include "nmtools/array/array/ufuncs/isfinite.hpp"
#include "nmtools/array/array/ufuncs/exp.hpp"
#include "nmtools/array/array/mean.hpp"
#include "nmtools/array/array/sum.hpp"
#include "nmtools/array/array/matmul.hpp"

#include "nmtools/array/array/reshape.hpp"
#include "nmtools/array/array/arange.hpp"

#include "nmtools/utils/to_string.hpp"

namespace nm = nmtools;
namespace na = nm::array;
namespace view = nm::view;
namespace meta = nm::meta;

using nm::utils::to_string, nm::None, nm::shape, nm::True, nm::False;
using namespace nm::literals;

In [3]:
#include <iostream>
#include <array>
#include <tuple>

using std::array, std::tuple;

#define PRINT_STR(x) \
std::cout << #x << ":\n" \
    << x \
    << std::endl;

#define DESC(x) std::cout << x << std::endl;
#define PRINT(x) \
std::cout << #x << ":\n" \
    << nm::utils::to_string(x) \
    << std::endl;

In [4]:
#if __has_include(<boost/type_index.hpp>)
    #include <boost/type_index.hpp>
    #define NMTOOLS_TESTING_HAS_TYPE_INDEX
#endif

#ifdef NMTOOLS_TESTING_HAS_TYPE_INDEX
#define NMTOOLS_GET_TYPENAME(type) \
boost::typeindex::type_id<type>().pretty_name()
#else
#define NMTOOLS_GET_TYPENAME(type) \
typeid(type).name()
#endif

prepare some data for demo

In [5]:
auto a = na::arange(12_ct, nm::float32);
auto o = meta::as_value_v<na::fixed_ndarray<float,2,3,2>>;
auto b = na::reshape(a, std::tuple{2_ct,3_ct,2_ct}, /*context=*/None, /*output=*/o);
PRINT(a);
PRINT(b);
auto a_type = NMTOOLS_GET_TYPENAME(decltype(a));
auto b_type = NMTOOLS_GET_TYPENAME(decltype(b));
PRINT_STR(a_type);
PRINT_STR(b_type);

a:
[	0.000000	1.000000	2.000000	3.000000	4.000000	5.000000	6.000000	7.000000	8.000000	9.000000	10.000000	11.000000]
b:
[[[	0.000000	1.000000],
[	2.000000	3.000000],
[	4.000000	5.000000]],

[[	6.000000	7.000000],
[	8.000000	9.000000],
[	10.000000	11.000000]]]
a_type:
nmtools::array::dynamic_ndarray<float, std::vector, std::vector>
b_type:
nmtools::array::fixed_ndarray<float, 2ul, 3ul, 2ul>


## Built-in Ufuncs

#### Broadcasting ufuncs

Broadcasting is enable for arity > 1. There is nothing to broadcast to for unary ufuncs. These kind of ufuncs doesn't takes additional arguments other than the operands.

In [6]:
{
    auto b_mul_3 = view::multiply(b, 3);
    auto type = NMTOOLS_GET_TYPENAME(decltype(b_mul_3));
    PRINT(b_mul_3);
    PRINT_STR(type);
}

b_mul_3:
[[[	0.000000	3.000000],
[	6.000000	9.000000],
[	12.000000	15.000000]],

[[	18.000000	21.000000],
[	24.000000	27.000000],
[	30.000000	33.000000]]]
type:
nmtools::view::decorator_t<nmtools::view::ufunc_t, nmtools::view::multiply_t<nmtools::none_t, nmtools::none_t, nmtools::none_t, void>, nmtools::view::decorator_t<nmtools::view::broadcast_to_t, nmtools::array::fixed_ndarray<float, 2ul, 3ul, 2ul>, std::tuple<std::integral_constant<unsigned long, 2ul>, std::integral_constant<unsigned long, 3ul>, std::integral_constant<unsigned long, 2ul> >, nmtools::array::hybrid_ndarray<unsigned long, 3ul, 1ul> >, nmtools::view::decorator_t<nmtools::view::broadcast_to_t, int, std::tuple<std::integral_constant<unsigned long, 2ul>, std::integral_constant<unsigned long, 3ul>, std::integral_constant<unsigned long, 2ul> >, nmtools::none_t> >


In [7]:
{
    auto exp = view::exp(b);
    auto type = NMTOOLS_GET_TYPENAME(decltype(exp));
    PRINT(exp);
    PRINT_STR(type);
}

exp:
[[[	1.000000	2.718282],
[	7.389056	20.085537],
[	54.598148	148.413162]],

[[	403.428802	1096.633179],
[	2980.958008	8103.083984],
[	22026.464844	59874.140625]]]
type:
nmtools::view::decorator_t<nmtools::view::ufunc_t, nmtools::view::exp_t, nmtools::array::fixed_ndarray<float, 2ul, 3ul, 2ul> >


Example of ufunc with arity of 3:

In [8]:
{
    auto min = 3;
    auto max = array{5,1};
    auto clipped = view::clip(b,min,max);
    auto type = NMTOOLS_GET_TYPENAME(decltype(clipped));
    PRINT(clipped);
    PRINT_STR(type);
}

clipped:
[[[	3.000000	1.000000],
[	3.000000	1.000000],
[	4.000000	1.000000]],

[[	5.000000	1.000000],
[	5.000000	1.000000],
[	5.000000	1.000000]]]
type:
nmtools::view::decorator_t<nmtools::view::ufunc_t, nmtools::view::clip_t, nmtools::view::decorator_t<nmtools::view::broadcast_to_t, nmtools::array::fixed_ndarray<float, 2ul, 3ul, 2ul>, std::tuple<std::integral_constant<unsigned long, 2ul>, std::integral_constant<unsigned long, 3ul>, std::integral_constant<unsigned long, 2ul> >, nmtools::array::hybrid_ndarray<unsigned long, 3ul, 1ul> >, nmtools::view::decorator_t<nmtools::view::broadcast_to_t, int, std::tuple<std::integral_constant<unsigned long, 2ul>, std::integral_constant<unsigned long, 3ul>, std::integral_constant<unsigned long, 2ul> >, nmtools::none_t>, nmtools::view::decorator_t<nmtools::view::broadcast_to_t, std::array<int, 2ul>, std::tuple<std::integral_constant<unsigned long, 2ul>, std::integral_constant<unsigned long, 3ul>, std::integral_constant<unsigned long, 2ul> >, nmtools

#### Reduce ufuncs

Reduce ufuncs only takes single operands, but with additional arguments such as axis and keepdims.

In [9]:
{
    auto flat_sum = view::sum(b, /*axis=*/None);
    auto axis2_sum = view::sum(b, 2); 
    PRINT(flat_sum);
    PRINT(axis2_sum);
}

flat_sum:
66.000000
axis2_sum:
[[	1.000000	5.000000	9.000000],
[	13.000000	17.000000	21.000000]]


In [10]:
{
    auto flat_mean = view::mean(b, None);
    auto flat_mean_keepdims = view::mean(b, None, /*dtype=*/None, /*keepdims=*/True);
    auto axis0_mean = view::mean(b, 0);
    auto axis1_mean = view::mean(b, 1);
    auto axis12_mean = view::mean(b, std::array{1,2});
    PRINT(flat_mean);
    PRINT(flat_mean_keepdims);
    PRINT(axis0_mean);
    PRINT(axis1_mean);
    PRINT(axis12_mean);
}

flat_mean:
5.500000
flat_mean_keepdims:
[[[	5.500000]]]
axis0_mean:
[[	3.000000	4.000000],
[	5.000000	6.000000],
[	7.000000	8.000000]]
axis1_mean:
[[	2.000000	3.000000],
[	8.000000	9.000000]]
axis12_mean:
[	2.500000	8.500000]


#### Accumulate ufuncs

Similar to reduce ufunc, accumulate ufuncs takes axis arguments. But unlike reduce ufuncs, accumulate ufuncs don't take keepdims arguments, the resulting arguments will always be the same.

In [11]:
{
    auto accumulated = view::accumulate_add(b, /*axis=*/1, None);
    PRINT(accumulated);
}

accumulated:
[[[	0.000000	1.000000],
[	2.000000	4.000000],
[	6.000000	9.000000]],

[[	6.000000	7.000000],
[	14.000000	16.000000],
[	24.000000	27.000000]]]


nmtools doesn't have generalized ufuncs, some function are implemented with built-in view.

In [12]:
{
    auto c = na::reshape(
        na::arange(12_ct, nm::float32),
        tuple{2_ct,6_ct}, /*context=*/None,
        /*output=*/meta::as_value_v<na::fixed_ndarray<float,2,6>>
    );
    auto matmul = view::matmul(b,c);
    auto type = NMTOOLS_GET_TYPENAME(decltype(matmul));
    PRINT(c);
    PRINT(matmul);
    PRINT_STR(type);
}

c:
[[	0.000000	1.000000	2.000000	3.000000	4.000000	5.000000],
[	6.000000	7.000000	8.000000	9.000000	10.000000	11.000000]]
matmul:
[[[	6.000000	7.000000	8.000000	9.000000	10.000000	11.000000],
[	18.000000	23.000000	28.000000	33.000000	38.000000	43.000000],
[	30.000000	39.000000	48.000000	57.000000	66.000000	75.000000]],

[[	42.000000	55.000000	68.000000	81.000000	94.000000	107.000000],
[	54.000000	71.000000	88.000000	105.000000	122.000000	139.000000],
[	66.000000	87.000000	108.000000	129.000000	150.000000	171.000000]]]
type:
nmtools::view::decorator_t<nmtools::view::matmul_t, nmtools::array::fixed_ndarray<float, 2ul, 3ul, 2ul>, nmtools::array::fixed_ndarray<float, 2ul, 6ul> >


## Custom Ufuncs

You can create custom operation and plug it to nmtools ufunc:

In [13]:
struct my_relu6
{
    template <typename T>
    auto operator()(const T& t) const
    {
        return static_cast<T>(t > 0 ? (t < 6 ? t : 6) : 0);
    }
};

In [14]:
{
    auto mrelu = my_relu6{};
    auto relu_out = view::ufunc(mrelu, b);
    auto relu_out_type = NMTOOLS_GET_TYPENAME(decltype(relu_out));
    PRINT(relu_out);
    PRINT_STR(relu_out_type);
}

relu_out:
[[[	0.000000	1.000000],
[	2.000000	3.000000],
[	4.000000	5.000000]],

[[	6.000000	6.000000],
[	6.000000	6.000000],
[	6.000000	6.000000]]]
relu_out_type:
nmtools::view::decorator_t<nmtools::view::ufunc_t, __cling_N512::my_relu6, nmtools::array::fixed_ndarray<float, 2ul, 3ul, 2ul> >


For operation with arity > 1, broadcasting will be applied:

In [15]:
struct a_plus_b_minus_1
{
    template <typename T, typename U>
    auto operator()(const T& t, const U& u) const
    {
        return static_cast<T>(t + u - 1);
    }
};

In [16]:
{
    auto op = a_plus_b_minus_1{};
    auto res = view::ufunc(op, b, 9);
    PRINT(res);
}

res:
[[[	8.000000	9.000000],
[	10.000000	11.000000],
[	12.000000	13.000000]],

[[	14.000000	15.000000],
[	16.000000	17.000000],
[	18.000000	19.000000]]]


Reduce ufunc need op with arity exacly 2:

In [17]:
{
    auto op = a_plus_b_minus_1{};
    auto res = view::reduce(op, b, /*axis=*/1, /*initial=*/None, /*keepdims=*/False);
    PRINT(res);
}

res:
[[	4.000000	7.000000],
[	22.000000	25.000000]]


Accumulate ufunc also need op with arity exactly 2, but without keepdims:

In [18]:
{
    auto op = a_plus_b_minus_1{};
    auto res = view::accumulate(op, b, /*axis=*/1);
    PRINT(res);
}

res:
[[[	0.000000	1.000000],
[	1.000000	3.000000],
[	4.000000	7.000000]],

[[	6.000000	7.000000],
[	13.000000	15.000000],
[	22.000000	25.000000]]]


## Lazy Eval vs. Eager Eval

The examples above only show lazy evaluation so far. In nmtools, lazy evaluation functions are under the namespace `nmtools::view`, while eager eval functions are under the namespace `nmtools::array`.

Lazy eval view perform the computation on demand on `operator()`. When the function called, the returned object is some kind of a proxy to the underlying array. The consequence is when the underlying array is changed, the "result" of the view also changed. Such changes don't happen for eager eval functions. Under the hood, the eager eval functions are implemented by creating a view and then calls `eval`.

In [19]:
{
    int a[2][3][2] = {
        {
            {0,1},
            {2,3},
            {4,5},
        },
        {
            { 6, 7},
            { 8, 9},
            {10,11},
        }
    };
    int b = 3;
    auto lazy_a_plus_b = view::add(a,b);
    auto eager_a_plus_b = na::add(a,b);
    auto lazy_reduced_a = view::reduce_add(a,/*axis=*/1);
    auto eager_reduced_a = na::add.reduce(a,/*axis=*/1);
    PRINT(lazy_a_plus_b);
    PRINT(eager_a_plus_b);
    PRINT(lazy_reduced_a);
    PRINT(eager_reduced_a);
    
    
    DESC("\n\nmodyfing the underlying array:");
    nm::at(a,0,1,0) = 99;
    PRINT(lazy_a_plus_b);
    PRINT(eager_a_plus_b);
    PRINT(lazy_reduced_a);
    PRINT(eager_reduced_a);
}

lazy_a_plus_b:
[[[	3	4],
[	5	6],
[	7	8]],

[[	9	10],
[	11	12],
[	13	14]]]
eager_a_plus_b:
[[[	3	4],
[	5	6],
[	7	8]],

[[	9	10],
[	11	12],
[	13	14]]]
lazy_reduced_a:
[[	6	9],
[	24	27]]
eager_reduced_a:
[[	6	9],
[	24	27]]


modyfing the underlying array:
lazy_a_plus_b:
[[[	3	4],
[	102	6],
[	7	8]],

[[	9	10],
[	11	12],
[	13	14]]]
eager_a_plus_b:
[[[	3	4],
[	5	6],
[	7	8]],

[[	9	10],
[	11	12],
[	13	14]]]
lazy_reduced_a:
[[	103	9],
[	24	27]]
eager_reduced_a:
[[	6	9],
[	24	27]]


The same also applies to custom ufuncs:

In [20]:
{
    int a[2][3][2] = {
        {
            {0,1},
            {2,3},
            {4,5},
        },
        {
            { 6, 7},
            { 8, 9},
            {10,11},
        }
    };
    int b = 3;
    auto eager = [](const auto& a, const auto& b){
        auto op = a_plus_b_minus_1{};
        auto v = view::ufunc(op, a, b);
        auto res = na::eval(v);
        return res;
    };
    // custom op
    auto op = a_plus_b_minus_1{};
    auto lazy_res = view::ufunc(op, a, b);
    auto eager_res = eager(a,b);
    
    PRINT(lazy_res);
    PRINT(eager_res);
    
    DESC("\n\nmodyfing the underlying array:");
    nm::at(a,0,1,0) = 99;
    
    PRINT(lazy_res);
    PRINT(eager_res);
}

lazy_res:
[[[	2	3],
[	4	5],
[	6	7]],

[[	8	9],
[	10	11],
[	12	13]]]
eager_res:
[[[	2	3],
[	4	5],
[	6	7]],

[[	8	9],
[	10	11],
[	12	13]]]


modyfing the underlying array:
lazy_res:
[[[	2	3],
[	101	5],
[	6	7]],

[[	8	9],
[	10	11],
[	12	13]]]
eager_res:
[[[	2	3],
[	4	5],
[	6	7]],

[[	8	9],
[	10	11],
[	12	13]]]
